In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append('../../enfify')

import os
from glob import glob
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import yaml
from analyses import classification_dacasil
from feature_processing import get_hilbert_phase
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from utils import read_wavfile

In [ ]:
with open("../../enfify/defaults.yml", "r") as f:
    config = yaml.safe_load(f)
config["downsample"]["is_enabled"] = False

In [ ]:
authentic_paths = sorted(glob("../../data/raw/synthetic/uncut/train_audio/*.wav"))
tampered_paths = sorted(glob("../../data/raw/synthetic/cut/train_cut_audio/*.wav"))

data_paths = authentic_paths + tampered_paths

In [ ]:
results = []
labels = []
for auth_path, tamp_path in zip((pbar:=tqdm(authentic_paths)), tampered_paths):
    for label, path in zip([True, False], [auth_path, tamp_path]):
        sig, fs = read_wavfile(path)
        x, phase = get_hilbert_phase(sig, fs, config)
        results.append(classification_dacasil(phase, x))
        labels.append(label)
    pbar.set_postfix_str(f"Accuracy: {accuracy_score(labels, results)*100:.2f} % ")

print(f"Total Accuracy: {accuracy_score(labels, results)}")

# pprint(list(zip(labels, results)))